In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [3]:
# 필요한 모듈 임포트
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool, AgentExecutor, create_react_agent
from datetime import datetime

In [4]:
# 1. 오늘 날짜 확인용 함수
def get_today():
    return datetime.today().strftime('%Y-%m-%d')

# 오늘 날짜 확인 도구 생성
def get_today_tool(input: str) -> str:
    return get_today()

today_tool = Tool(
    name="Get Today",
    func=get_today_tool,
    description="Returns today's date in YYYY-MM-DD format",
    verbose=True
)

In [5]:
# 2. Google Search Wrapper 생성
google_search = GoogleSerperAPIWrapper()

In [6]:
# 3. 모델(LLM) 생성
llm = ChatOpenAI(
    temperature=0.1,
    max_tokens=2048,
    model_name="gpt-4o-mini"  # 사용할 GPT 모델 설정
)

In [60]:
## : 문서 로드(Load Documents)
loader = PyMuPDFLoader("stock_report/[HD현대]분기보고서(2024.11.14).pdf")
docs = loader.load()

In [61]:
## : 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [62]:
## 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [63]:
## 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [64]:
# 5. 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [65]:
# RAG 도구 생성
def rag_tool(input: str) -> str:
    docs = retriever.get_relevant_documents(input)
    
    # 예외 처리: 검색 결과가 없거나 비어있을 때
    if not docs:  # None, Empty List 모두 처리
        print("[RAG Search] 검색 결과가 없습니다. 다음 질문을 시도해주세요.")
        return "검색 결과가 없습니다. 더 구체적인 질문을 해주세요."
    
    # 예외 처리: 검색 결과가 있으나 내용이 비어 있을 때
    if all(not doc.page_content.strip() for doc in docs):
        print("[RAG Search] 검색 결과는 있으나 내용이 비어 있습니다.")
        return "관련된 내용이 없습니다. 다른 질문을 시도해주세요."
    
    return "\n".join([doc.page_content for doc in docs])

rag_search_tool = Tool(
    name="RAG Search",
    func=rag_tool,
    description="Useful for when you need to search from the semi-annual report",
    verbose=True
)

In [66]:
# 6. tools 리스트에 추가
tools = [
    Tool(
        name="Google Search",
        func=google_search.run,
        description="Useful for when you need to search with Google",
        verbose=True
    ),
    today_tool,
    rag_search_tool
]

In [67]:
# tools 이름 리스트 업데이트
tool_names = ", ".join([tool.name for tool in tools])

In [76]:
# 7. PromptTemplate 정의
template = '''You are a skilled financial analyst.

1. Analyze the Question and think about the necessary information required to provide the Final Answer.
2. Use the thought process to formulate the Action Input.
3. Use the RAG Search tool to retrieve relevant information from the vector store.
4. Only use Google Search if the information cannot be found in the vector store.
5. If the desired result is not obtained, modify the Action Input and continue searching for information.
6. The Final Answer must be written in Korean.
7. RAG Search should not search with the same Action Input repeatedly.:

Tools available:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

In [77]:
# 8. PromptTemplate에 tool_names 전달
prompt = PromptTemplate.from_template(template).partial(
    tool_names=tool_names,
    tools="\n".join([f"- {tool.name}: {tool.description}" for tool in tools])
)

In [78]:
# 9. React Agent 생성
search_agent = create_react_agent(llm, tools, prompt)

In [79]:
# 10. Agent Executor 생성
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [80]:
# 11. 입력 질문을 통해 Agent 실행
response = agent_executor.invoke({
    "input": "현대hd 주식의 적정가 책정해봐.",
    "today_date": get_today()  # 오늘 날짜를 추가
})

# 12. 결과 출력
print(response)



> Entering new AgentExecutor chain...
주식의 적정가를 책정하기 위해서는 현대HD의 재무제표, 최근 실적, 산업 동향, 경쟁사 분석, 그리고 시장의 전반적인 상황 등을 고려해야 합니다. 이러한 정보는 반기 보고서에서 찾을 수 있을 것으로 보입니다. 

Action: RAG Search  
Action Input: 현대HD 반기 보고서  
로고와 “HD”를 출원하였으며, 보고서 기준일 현재 HD현대㈜가 해당 상표권을 단독으로 소유하고 있
습니다. 
 
나. 사업부문별 재무정보 및 부문비중 
(단위 : 백만원, %)
구         분
제8기 3분기
제7기
제 6 기
비고
금액
비중
금액
비중
금액
비중
조선해양 부문
1. 매    출    액
20,340,337
40.1
23,818,128
38.8
16,945,467
27.8
2. 영  업  손  익
985,252
46.8
332,186
16.4
118,963
3.5
3. 자  산  총  계
32,026,087
47.1
32,242,568
47.4
39,554,055
60.3
(감가상각비)
(383,677)
27.3
(475,060)
27.0
(357,476)
22.6
정유 부문
　
　
1. 매    출    액
36,774,522
72.6
39,257,078
64.0
49,155,655
80.8
2. 영  업  손  익
126,339
6.0
611,770
30.1
2,784,579
82.2
3. 자  산  총  계
19,307,063
28.4
19,668,994
28.9
22,387,360
34.1
(감가상각비)
(667,916)
47.5
(868,471)
49.4
(791,612)
50.0
전기전자 부문
　
　
1. 매    출    액
3,241,406
6.4
3,371,264
5.5
2,540,746
4.2
2. 영  업  손  익
519,157
24.7
362,165
17.8
144,702
4.3
3. 자  산  총  계
3,365,665
4.9
2,910,160
4.